# 감정 분석

In [1]:
!pip install gluonnlp pandas tqdm
!pip install mxnet
!pip install sentencepiece

!pip install transformers
!pip install torch
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661679 sha256=1760a0ac9c635aa2936df1b1a80cbc37256fe68cfc74257ccfffba87c3ce0baf
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 14.2 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.8 MB/s eta 0:00:00
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-bw_xcmw3/kobert-tokenizer_5f4a1005ebd740438fc781f8d5ab89bc
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-bw_xcmw3/kob

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from sklearn.model_selection import train_test_split
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel,AutoConfig
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

cuda:0


## Setting Parameters

In [5]:
# Setting parameters
max_len = 100
batch_size = 64
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [6]:
from tqdm import tqdm, tqdm_notebook
from glob import glob
import gc
import os

## BERT Dataset

In [7]:
#gluonnlp의 BERTSentenceTransform을 사용하여 입력 문장을 BERT 입력 형식에 맞게 변환 -transform 메서드를 사용하여 각 문장을 BERT 입력 형식에 맞게 변환
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

## BERT Classifier

In [8]:
config=AutoConfig.from_pretrained('skt/kobert-base-v1')
config

config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

BertConfig {
  "_name_or_path": "skt/kobert-base-v1",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "kobert_version": 1.0,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.35.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 8002
}

In [9]:
#입력 데이터의 패딩 부분을 제외하고, 실제 입력에 대한 어텐션 마스크를 생성하는 함수

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier =  nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=hidden_size, out_features=512),
            nn.Linear(in_features=512, out_features=num_classes)
        ) #nn.Linear(hidden_size , num_classes)

        #정규화 레이어 추가 (Layernormalization)
        self.layer_norm = nn.LayerNorm(768)

        #드롭아웃
        self.dropout = nn.Dropout(p=dr_rate)


    def gen_attention_mask(self, token_ids, valid_length): #token_ids는 입력 문장을 토큰화한 결과
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length) #gen_attention_mask 메서드를 사용하여 어텐션 마스크를 생성

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device)) #BERT 모델에 입력을 전달하여 출력을 계산

        pooled_output = self.dropout(pooler)
        normalized_output = self.layer_norm(pooled_output)  # Layer Normalization 적용
        out=self.classifier(normalized_output)

        return out


## Tokenizer & Model 정의

In [10]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1') #skt/kobert-base-v1 모델의 사전 학습된 가중치 로드
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]') ##BERTVocab 객체에 tokenizer.vocab_file 사전(vocab) 로드
tok = tokenizer.tokenize

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [11]:
#모델 구조 확인
BERTClassifier(bertmodel, dr_rate=0.5)

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

## 데이터 로드 및 전처리
* 데이터셋 출처 : https://aihub.or.kr/mypage/reqst/datareqst/view.do?currMenu=133&topMenu=105&dataReqstSn=378435

In [12]:
dd=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/감성대화말뭉치(최종데이터)_Validation.csv')
dd.head()

,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,1,청년,남성,"진로,취업,직장",해당없음,불안,두려운,이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너...,실수하시다니 정말 미안한 마음이 크겠어요.,내 능력이 부족한 거 같은데 그만 다녀야 될거같아.,능력을 올리려면 어떤 방법이 있을까요?,퇴근 후 여가에 회사 일을 더 열심히 해서 피해가 가지 않도록 해야겠어.,꼭 좋은 결과 있길 바라요.
1,2,청년,남성,"진로,취업,직장",해당없음,불안,두려운,회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워.,큰 프로젝트를 혼자 하셔서 고민이 많겠네요.,나에게 너무 크게 느껴지는 중요한 프로젝트라 버거운 느낌이 들어.,프로젝트를 잘하시기 위해서 어떤 걸 할 수 있나요?,동료 직원에게 도움을 요청해서 같이 해결해야겠어.,동료 직원에게 도움을 요청하기로 하셨군요.
2,3,청년,남성,"진로,취업,직장",해당없음,불안,두려운,상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워.,직장 상사가 무섭게 생기셔서 친해지는 게 어렵군요.,무섭게 생겼는데도 업무를 보려면 친해져야 할 것 같단말이야.,상사분과 친해질 수 있는 방법은 무엇이 있을까요?,먼저 다가가서 말을 걸어볼게.,직장 상사와 친해지시면 좋겠네요.
3,4,청년,남성,"진로,취업,직장",해당없음,불안,두려운,이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다.,첫 직장이라서 정말 떨리시고 긴장되실 것 같아요.,첫 직장이어서 잘 적응을 할 수 있을지 모르겠어.,잘 적응 하시려면 무엇을 할 수 있을까요?,직장 동료와 상사들이랑 친하게 지내야겠어.,직장에 잘 적응하시길 바라요.
4,5,청년,남성,"진로,취업,직장",해당없음,불안,두려운,직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼.,직장 사람들 관계에 대해서 고민이시군요.,내가 낯가림이 심해서 친해질 수 있을지 모르겠어.,직장 사람들과 친해지려면 무슨 방법이 있을까요?,서로 같은 취미를 공유하고 얘기를 나누다 보면 친해질 수 있을 거 같아.,직장 사람들과 좋은 관계를 가지시길 응원해요.


In [13]:
train_set = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/감성대화말뭉치(최종데이터)_Training.csv')
validation_set = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/감성대화말뭉치(최종데이터)_Validation.csv')
train_set = train_set.loc[:, ['감정_대분류', '사람문장1']]
validation_set = validation_set.loc[:, ['감정_대분류', '사람문장1']]


train_set.dropna(inplace=True)
validation_set.dropna(inplace=True)
train_set.columns = ['label', 'data']
validation_set.columns = ['label', 'data']

## 상처와 슬픔을 하나의 레이블로 설정

#분노, 불안, 상처, 기쁨, 슬픔, 당황
train_set.loc[(train_set['label'] == '불안'), 'label'] = 0
train_set.loc[(train_set['label'] == '분노'), 'label'] = 1
train_set.loc[(train_set['label'] == '상처'), 'label'] = 2
train_set.loc[(train_set['label'] == '슬픔'), 'label'] = 2
train_set.loc[(train_set['label'] == '당황'), 'label'] = 3
train_set.loc[(train_set['label'] == '기쁨'), 'label'] = 4
train_set['label']=train_set['label'].astype(int)


validation_set.loc[(validation_set['label'] == '불안'), 'label'] = 0
validation_set.loc[(validation_set['label'] == '분노'), 'label'] = 1
validation_set.loc[(validation_set['label'] == '상처'), 'label'] = 2
validation_set.loc[(validation_set['label'] == '슬픔'), 'label'] = 2
validation_set.loc[(validation_set['label'] == '당황'), 'label'] = 3
validation_set.loc[(validation_set['label'] == '기쁨'), 'label'] = 4
validation_set['label']=validation_set['label'].astype(int)

df=pd.concat([train_set,validation_set])
df.dropna(inplace=True)

train_set_data = [[i, str(j)] for i, j in zip(df['data'], df['label'])]

## validation_set_data = [[i, str(j)] for i, j in zip(validation_set['data'], validation_set['label'])]

train_set_data, test_set_data = train_test_split(train_set_data, test_size = 0.2, random_state=4)
train_set_data = BERTDataset(train_set_data, 0, 1, tok, vocab, max_len, True, False)
test_set_data = BERTDataset(test_set_data, 0, 1, tok, vocab, max_len, True, False)
train_dataloader = torch.utils.data.DataLoader(train_set_data, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_set_data, batch_size=batch_size, num_workers=2)


In [14]:
df['label'].value_counts() #클래스 불균형 -> 평가지표 : f1_score

2    20278
0    10433
1    10417
3     9804
4     7339
Name: label, dtype: int64

## 모델 학습

In [15]:
from sklearn.metrics import f1_score
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    #acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    f1score=f1_score(Y.data.cpu().numpy(),max_indices.data.cpu().numpy(),average='macro') #macro 평균은 클래스별 f1-점수에 가중치를 주지 않음. 클래스 크기에 상관없이 모든 클래스를 같은 비중으로 다룬다.
    return f1score

def predict(sentence):
    dataset = [[sentence, '0']]
    test = BERTDataset(dataset, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=2)
    model.eval()
    answer = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        for logits in out:
            logits = logits.detach().cpu().numpy()
            answer = np.argmax(logits)
    return answer

In [16]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight'] # 가중치 감쇠(weight decay)를 적용하여 과적합을 방지
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
] # 가중치 감쇠를 적용하지 않는 파라미터와 적용하는 파라미터를 구분하여 옵티마이저를 설정

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate) #AdamW 옵티마이저를 사용하여 학습을 진행
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train f1score {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train f1score {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test f1score {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-16-5ce332bcecc6>:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/729 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.6346133947372437 train f1score 0.20182011098450273
epoch 1 batch id 201 loss 1.5030759572982788 train f1score 0.14849068641358035
epoch 1 batch id 401 loss 1.1466549634933472 train f1score 0.24464554158617724
epoch 1 batch id 601 loss 1.1193196773529053 train f1score 0.3481131643525007
epoch 1 train f1score 0.3883116688306538


<ipython-input-16-5ce332bcecc6>:38: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/183 [00:00<?, ?it/s]

...

  0%|          | 0/729 [00:00<?, ?it/s]

epoch 20 batch id 1 loss 0.0011594787938520312 train f1score 1.0
epoch 20 batch id 201 loss 0.0025520918425172567 train f1score 0.9913176561399838
epoch 20 batch id 401 loss 0.0008264869684353471 train f1score 0.9909138948073479
epoch 20 batch id 601 loss 0.021691715344786644 train f1score 0.9906144724180043
epoch 20 train f1score 0.9903433376833997


<ipython-input-16-5ce332bcecc6>:38: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/183 [00:00<?, ?it/s]

epoch 20 test f1score 0.6267548125214074


In [18]:
#모델 저장
torch.save(model, '/content/drive/MyDrive/Colab Notebooks/CustomKoBERTWithLayNorm_epoch20_F1.pth')

#모델 저장
#torch.save(model, '/content/drive/MyDrive/Colab Notebooks/CustomKoBERTWithLayNorm_epoch20.pth')

#모델 불러오기
# 모델 호출
#my_model_mark4 = torch.load('/content/drive/MyDrive/Colab Notebooks/customkobert_epoch50.pth')

## Test

In [24]:
diary='''
오늘은 아빠가 집에 돌아오는 날이다. 아빠랑 같이 치킨먹으면서 영화볼 생각하니까 설렌다
'''

emotion_arr=['불안','분노','슬픔','당황','기쁨']
emotion=predict(diary)
print(emotion_arr[emotion])

기쁨


In [30]:
diary='''
오늘 엄마랑 싸웠다..
'''
emotion_arr=['불안','분노','슬픔','당황','기쁨']
emotion=predict(diary)
print(emotion_arr[emotion])

슬픔


In [20]:
diary2='''
오늘은 예상치 못한 일들로 인해 마음이 혼란스러운 날이었다.
아침에 시작된 하루는 어딘가에서부터 조금 이상했다. 일상의 흐름이 달라진 것 같았고, 그 변화에 따라 마음도 어느새 혼란스러워졌다.
시간이 흘러가는 게 이상하게 느껴졌고, 계획한 것과 실제로 이루어지는 것 사이에 어긋남이 있었다. 일상의 일들이 마치 뒤섞이는 듯한 기분이었다.
정신을 가다듬고 일상을 되짚어보려고 했지만, 마음이 급속도로 얽혀져만 갔다.하루 중간에는 내가 기억하고 있던 것들과 현실이 서로 다르게 느껴졌다.
이상한 일들이 계속해서 일어나면서 마음은 점점 더 혼란스러워져만 갔다. 마치 하루가 이상한 꿈을 꾸고 있는 것 같은 기분이었다.
그런데도 시간은 계속 흘러가고, 날이 저물어가는 게 느껴졌다. 마음의 혼란은 여전하지만, 내일이라는 새로운 시작에 대한 희망이 간신히 남아있다.
이런 날도 가끔 있는 법이지. 내일은 분명 더 나은 하루가 될 거야.
'''

emotion_arr=['불안','분노','슬픔','당황','기쁨']
emotion=predict(diary2)
print(emotion_arr[emotion])

불안
